ASSIGNMENT 4

In [60]:
import pandas as pd
import numpy as np

In [65]:
train_data = pd.read_json("C:/Users/tejas/OneDrive/Desktop/Assignments/COMPSCI 753/goodreads_reviews_historybio_train.json", lines=True)
test_data = pd.read_json("C:/Users/tejas/OneDrive/Desktop/Assignments/COMPSCI 753/goodreads_reviews_historybio_test.json", lines=True)
val_data = pd.read_json("C:/Users/tejas/OneDrive/Desktop/Assignments/COMPSCI 753/goodreads_reviews_historybio_val.json", lines=True)

Task 1 - Calculate the global bias, user specific bias and item specific bias on the training data.

(A) Report the global bias

In [44]:
global_bias = train_data['rating'].mean()
print("Global bias:", global_bias)

Global bias: 3.7669762808387413


(B) Report the user specific bias of user id= “3913f3be1e8fadc1de34dc49dab06381”

In [45]:
user_id = "3913f3be1e8fadc1de34dc49dab06381"
user_bias = train_data[train_data['user_id'] == user_id]['rating'].mean() - global_bias
print("User-specific bias for user id =", user_id, ":", user_bias)

User-specific bias for user id = 3913f3be1e8fadc1de34dc49dab06381 : -0.1139150563489455


(C) Report the item specific bias of book id = “16130”

In [46]:
book_id = 16130
item_bias = train_data[train_data['book_id'] == book_id]['rating'].mean() - global_bias
print("Item-specific bias for book id =", book_id, ":", item_bias)

Item-specific bias for book id = 16130 : 0.4562653093753264


Task 2 - Implement the regularized latent factor model
without bias using SGD

(A) Implement the regularized latent factor model without considering the bias.

In [50]:
np.random.seed(123)

k = 8 
learning_rate = 0.01
lambda1 = 0.3
lambda2 = 0.3
epochs = 10

user_id_to_index = {user_id: idx for idx, user_id in enumerate(train_data['user_id'].unique())}
book_id_to_index = {book_id: idx for idx, book_id in enumerate(train_data['book_id'].unique())}

# Initialize matrices P and Q 
num_users = len(user_id_to_index)
num_items = len(book_id_to_index)

P = np.random.normal(size=(num_users, k))
Q = np.random.normal(size=(num_items, k))

# SGD
for epoch in range(epochs):
    total_error = 0

    for _, row in train_data.iterrows():
        user_id = row['user_id']
        book_id = row['book_id']
        rating = row['rating']

        # Convert user_id and book_id to indices
        i = user_id_to_index[user_id]
        j = book_id_to_index[book_id]

        # Prediction
        prediction = np.dot(Q[j], P[i])

        # Error
        error = rating - prediction
        total_error += error ** 2

        # Update P and Q using SGD
        P[i] += learning_rate * (error * Q[j] - lambda1 * P[i])
        Q[j] += learning_rate * (error * P[i] - lambda2 * Q[j])

    # Calculate RMSE for the epoch
    rmse = np.sqrt(total_error / len(train_data['rating']))
    print(f"Epoch {epoch + 1}/{epochs}, RMSE: {rmse}")

Epoch 1/10, RMSE: 4.456335932717587
Epoch 2/10, RMSE: 3.9524613388510756
Epoch 3/10, RMSE: 3.6863294126107085
Epoch 4/10, RMSE: 3.3672032569617394
Epoch 5/10, RMSE: 3.036559279113996
Epoch 6/10, RMSE: 2.748625750811755
Epoch 7/10, RMSE: 2.506139704724196
Epoch 8/10, RMSE: 2.3010319089911686
Epoch 9/10, RMSE: 2.126057000024525
Epoch 10/10, RMSE: 1.9757311913745244


(B) Use SGD to train the latent factor model on the training data for different
values of k in {4, 8, 16}. For each value of k, train the model for 10 epoches/iterations.
Report the RMSE for each value of k on the validation data. Pick the model that results
in the best RMSE on the validation set and report its RMSE on the test data.

In [22]:
learning_rate = 0.01
lambda1 = 0.3
lambda2 = 0.3
epochs = 10

# Values of k to try
k_values = [4, 8, 16]

best_model = None
best_rmse = float('inf')

for k in k_values:
    # Create mappings for user_id and book_id to indices
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(train_data['user_id'].unique())}
    book_id_to_index = {book_id: idx for idx, book_id in enumerate(train_data['book_id'].unique())}

    # Initialize matrices P and Q
    num_users = len(user_id_to_index)
    num_items = len(book_id_to_index)
    P = np.random.normal(size=(num_users, k))
    Q = np.random.normal(size=(num_items, k))

    # SGD
    for epoch in range(epochs):
        total_error = 0

        for _, row in train_data.iterrows():
            user_id = row['user_id']
            book_id = row['book_id']
            rating = row['rating']

            # Convert user_id and book_id to indices
            i = user_id_to_index[user_id]
            j = book_id_to_index[book_id]

            # Prediction
            prediction = np.dot(Q[j], P[i])

            # Error
            error = rating - prediction
            total_error += error ** 2

            # Update P and Q using SGD
            P[i] += learning_rate * (error * Q[j] - lambda1 * P[i])
            Q[j] += learning_rate * (error * P[i] - lambda2 * Q[j])

        # Calculate RMSE for the epoch on validation data
        val_rmse = np.sqrt(total_error / len(train_data))  # Adjusted this line

        print(f"Epoch {epoch + 1}/{epochs}, k={k}, Validation RMSE: {val_rmse}")

    # If this model has the best RMSE on the validation set, update the best model
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        best_model = (P.copy(), Q.copy())

    # Calculate RMSE for the test data after all epochs
    total_error_test = 0
    num_test_samples = 0

    for _, row in test_data.iterrows():
        user_id = row['user_id']
        book_id = row['book_id']
        rating = row['rating']

    # Check if user_id and book_id exist in the dictionary
        if user_id in user_id_to_index and book_id in book_id_to_index:
            # Convert user_id and book_id to indices
            i = user_id_to_index[user_id]
            j = book_id_to_index[book_id]

        # Prediction
            prediction = np.dot(Q[j], P[i])

        # Error
            error = rating - prediction
            total_error_test += error ** 2
            num_test_samples += 1

# Calculate RMSE for the test data
test_rmse = np.sqrt(total_error_test / num_test_samples) if num_test_samples > 0 else float('inf')
print(f"\nBest Model - Test RMSE: {test_rmse}")

Epoch 1/10, k=4, Validation RMSE: 4.218897033133351
Epoch 2/10, k=4, Validation RMSE: 3.9745931556423226
Epoch 3/10, k=4, Validation RMSE: 3.8475118967128497
Epoch 4/10, k=4, Validation RMSE: 3.676404752272193
Epoch 5/10, k=4, Validation RMSE: 3.424581718893356
Epoch 6/10, k=4, Validation RMSE: 3.1579736694340874
Epoch 7/10, k=4, Validation RMSE: 2.917251661430983
Epoch 8/10, k=4, Validation RMSE: 2.7079944567897156
Epoch 9/10, k=4, Validation RMSE: 2.5258700931223284
Epoch 10/10, k=4, Validation RMSE: 2.3660954105671834
Epoch 1/10, k=8, Validation RMSE: 4.454090582031795
Epoch 2/10, k=8, Validation RMSE: 3.9497254464254636
Epoch 3/10, k=8, Validation RMSE: 3.673209598582915
Epoch 4/10, k=8, Validation RMSE: 3.3435356956490976
Epoch 5/10, k=8, Validation RMSE: 3.01105507763816
Epoch 6/10, k=8, Validation RMSE: 2.7250661887691967
Epoch 7/10, k=8, Validation RMSE: 2.4853194592133336
Epoch 8/10, k=8, Validation RMSE: 2.2829089104302254
Epoch 9/10, k=8, Validation RMSE: 2.110251673094692
E

Task 3 - Implement the regularized latent factor model
with bias using SGD

(A) Incorporate the bias terms bg, b(user)i and b(item)j to the latent factor model.

In [57]:
k = 8
learning_rate = 0.01
lambda1 = 0.3
lambda2 = 0.3
lambda3 = 0.3
lambda4 = 0.3
epochs = 10

# Create mappings for user_id and book_id to indices
user_id_to_index = {user_id: idx for idx, user_id in enumerate(train_data['user_id'].unique())}
book_id_to_index = {book_id: idx for idx, book_id in enumerate(train_data['book_id'].unique())}

# Initialize matrices P and Q
num_users = len(user_id_to_index)
num_items = len(book_id_to_index)
P = np.random.normal(size=(num_users, k))
Q = np.random.normal(size=(num_items, k))

# Initialize bias terms
user_bias = {user_id: 0 for user_id in train_data['user_id'].unique()}
item_bias = {book_id: 0 for book_id in train_data['book_id'].unique()}

# SGD
for epoch in range(epochs):
    total_error = 0

    for _, row in train_data.iterrows():
        user_id = row['user_id']
        book_id = row['book_id']
        rating = row['rating']

        # Convert user_id and book_id to indices
        i = user_id_to_index.get(user_id, -1)
        j = book_id_to_index.get(book_id, -1)

        # Check for new users or items
        if i == -1 or j == -1:
            user_bias_i = 0
            item_bias_j = 0
        else:
            user_bias_i = user_bias[user_id]
            item_bias_j = item_bias[book_id]

        # Prediction
        prediction = np.dot(Q[j], P[i]) + user_bias_i + item_bias_j

        # Error
        error = rating - prediction
        total_error += error ** 2

        # Update P, Q, user_bias, and item_bias using SGD
        P[i] += learning_rate * (error * Q[j] - lambda1 * P[i])
        Q[j] += learning_rate * (error * P[i] - lambda2 * Q[j])
        user_bias[user_id] += learning_rate * (error - lambda3 * user_bias[user_id])
        item_bias[book_id] += learning_rate * (error - lambda4 * item_bias[book_id])

    # Calculate RMSE for the epoch
    rmse = np.sqrt(total_error / len(train_data))
    print(f"Epoch {epoch + 1}/{epochs}, RMSE: {rmse}")

# Report learned biases
user_id_to_find = "3913f3be1e8fadc1de34dc49dab06381"
book_id_to_find = 16130

learned_user_bias = user_bias.get(user_id_to_find, "Not Found")
learned_item_bias = item_bias.get(book_id_to_find, "Not Found")

print(f"\nLearned user-specific bias for {user_id_to_find}: {learned_user_bias}")
print(f"Learned item-specific bias for {book_id_to_find}: {item_bias.get(book_id_to_find, 'Not Found')}")

Epoch 1/10, RMSE: 3.551357939080106
Epoch 2/10, RMSE: 2.4030778255372804
Epoch 3/10, RMSE: 1.9805711925485754
Epoch 4/10, RMSE: 1.7396813492496281
Epoch 5/10, RMSE: 1.584403212028687
Epoch 6/10, RMSE: 1.4773995163558475
Epoch 7/10, RMSE: 1.4001880225210526
Epoch 8/10, RMSE: 1.3424124063671823
Epoch 9/10, RMSE: 1.297822498128653
Epoch 10/10, RMSE: 1.2624550166995512

Learned user-specific bias for 3913f3be1e8fadc1de34dc49dab06381: 2.5096203616916104
Learned item-specific bias for 16130: 2.382003855945663


(B) Similar to Task 2 (B), find the best k in {4, 8, 16} for the model you devel-
oped in Task 3 (A) on the validation set, by using RMSE to compare across these models,
and apply the best of these models to the test data. Compare the resulting test RMSE with
Task 2 (B). Analyse and explain your findings.

In [68]:
#sample_size = 15
#train_data = train_data.sample(n=sample_size, random_state=42)
#test_data = test_data.sample(n=sample_size, random_state=42)
#val_data = val_data.sample(n=sample_size, random_state=42)

# Define hyperparameters
learning_rate = 0.01
lambda1 = 0.3
lambda2 = 0.3
lambda3 = 0.3
lambda4 = 0.3
epochs = 10

# Candidate values for k
candidate_ks = [4, 8, 16]

# Create mappings for user_id and book_id to indices
user_id_to_index = {user_id: idx for idx, user_id in enumerate(train_data['user_id'].unique())}
book_id_to_index = {book_id: idx for idx, book_id in enumerate(train_data['book_id'].unique())}

# Initialize matrices P and Q
num_users = len(user_id_to_index)
num_items = len(book_id_to_index)

best_k = None
best_rmse = float('inf')

for k in candidate_ks:
    # Initialize matrices P and Q for the current k
    P = np.random.normal(size=(num_users, k))
    Q = np.random.normal(size=(num_items, k))

    # Initialize bias terms
    user_bias = {user_id: 0 for user_id in train_data['user_id'].unique()}
    item_bias = {book_id: 0 for book_id in train_data['book_id'].unique()}

    # SGD
    for epoch in range(epochs):
        total_error = 0

        for _, row in train_data.iterrows():
            user_id = row['user_id']
            book_id = row['book_id']
            rating = row['rating']

            # Convert user_id and book_id to indices
            i = user_id_to_index.get(user_id, -1)
            j = book_id_to_index.get(book_id, -1)

            # Check for new users or items
            if i == -1 or j == -1:
                user_bias_i = 0
                item_bias_j = 0
            else:
                user_bias_i = user_bias[user_id]
                item_bias_j = item_bias[book_id]

            # Prediction
            prediction = np.dot(Q[j], P[i]) + user_bias_i + item_bias_j

            # Error
            error = rating - prediction
            total_error += error ** 2

            # Update P, Q, user_bias, and item_bias using SGD
            P[i] += learning_rate * (error * Q[j] - lambda1 * P[i])
            Q[j] += learning_rate * (error * P[i] - lambda2 * Q[j])
            user_bias[user_id] += learning_rate * (error - lambda3 * user_bias[user_id])
            item_bias[book_id] += learning_rate * (error - lambda4 * item_bias[book_id])

        # Calculate RMSE for the epoch
        rmse = np.sqrt(total_error / len(train_data))

    # Evaluate on the validation set
    val_predictions = []

    for _, row in val_data.iterrows():
        user_id = row['user_id']
        book_id = row['book_id']
        rating = row['rating']

        # Convert user_id and book_id to indices
        i = user_id_to_index.get(user_id, -1)
        j = book_id_to_index.get(book_id, -1)

        # Check for new users or items
        if i == -1 or j == -1:
            user_bias_i = 0
            item_bias_j = 0
        else:
            user_bias_i = user_bias[user_id]
            item_bias_j = item_bias[book_id]

        # Prediction
        prediction = np.dot(Q[j], P[i]) + user_bias_i + item_bias_j
        val_predictions.append(prediction)

    val_rmse = np.sqrt(total_error / len(val_data))
    print(f"Epoch {epoch + 1}/{epochs}, k={k}, Validation RMSE: {val_rmse}")

    # Update best_k if the current model is better
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        best_k = k

print(f"\nBest k: {best_k}, Best Validation RMSE: {best_rmse}")

# Use the best model on the test set
test_predictions = []

for _, row in test_data.iterrows():
    user_id = row['user_id']
    book_id = row['book_id']
    rating = row['rating']

    # Convert user_id and book_id to indices
    i = user_id_to_index.get(user_id, -1)
    j = book_id_to_index.get(book_id, -1)

    # Check for new users or items
    if i == -1 or j == -1:
        user_bias_i = 0
        item_bias_j = 0
    else:
        user_bias_i = user_bias[user_id]
        item_bias_j = item_bias[book_id]

    # Prediction
    prediction = np.dot(Q[j], P[i]) + user_bias_i + item_bias_j
    test_predictions.append(prediction)

test_rmse = np.sqrt(total_error / len(test_data))
print(f"Test RMSE using the best model (k={best_k}): {test_rmse}")

Epoch 10/10, k=4, Validation RMSE: 2.397853277523409
Epoch 10/10, k=8, Validation RMSE: 2.189841346103022
Epoch 10/10, k=16, Validation RMSE: 1.9654329964601407

Best k: 16, Best Validation RMSE: 1.9654329964601407
Test RMSE using the best model (k=16): 1.9654329964601407


The validation RMSE decreases as the value of k increases. This is a common behavior, as a more complex model (higher k) can better fit the training data. The model with k=16 achieves the lowest validation RMSE, indicating that it is the best-performing model among the tested values of k.The test RMSE using the best model (k=16) is reported as 1.9654, which is consistent with the validation RMSE. This consistency suggests that the model is generalizing well to unseen data.